# CADET Settings

This module has all of the standard pieces of code we need for the workshop so that we don't have to repeat it for every lesson.

## Standard imports

In [1]:
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.display import Image

#Python path library support
from pathlib import Path

#python numeric library
import numpy

#scientific library for python
import scipy

#pandas is python library for data analysis
import pandas

#addict is a library that makes it easier to create nested dictionaries
from addict import Dict

#json is a standard text based format and it used in CADETMatch for the configuration file
import json

%config InlineBackend.figure_format='svg'
%matplotlib inline

#python plotting library
import matplotlib.pyplot as plt

#cadet python interface
from cadet import Cadet

#jupyter widget support
from ipywidgets import interact, interactive
import ipywidgets as widgets

from CADETMatch.jupyter import Match

## Set path to CADET bin folder
The next step is to tell the system where CADET can be found.

If you have followed all the instructions above you can just run the following cell and it should automatically work.

In [2]:
#This import is used to detect what operating system is being used to make this code automatic
import platform

#put in the path to the bin folder
cadet_bin_path = Path.home() / "CADET" / "cadet" / "bin"

if platform.system() == 'Windows':
    cadet_path = cadet_bin_path / "cadet-cli.exe"
    lwe_path = cadet_bin_path / "createLWE.exe"
else:
    cadet_path = cadet_bin_path / "cadet-cli"
    lwe_path = cadet_bin_path / "createLWE"

if cadet_path.exists() and lwe_path.exists():
    print("CADET was found. Continue to the next step.")
    Cadet.cadet_path = cadet_path.as_posix()
elif cadet_path.exists() and not lwe_path.exists():
    print("CADET was found but createLWE.exe was not found. Please make sure that none of the files have been moved.")
else:
    print("CADET could not be found. Please check the bin path")

CADET was found. Continue to the next step.


## Create lesson folders and create lesson variables

This will create all the lesson folders we need and make it easier to refer to previous lessons

In [3]:
#set this to a directory on your computer where the cadet tutorial files can be saved
lesson_dir = Path.home() / "CADET" / "lessons"
install = lesson_dir / "lesson_0"
lesson_1 = lesson_dir / "lesson_1"
lesson_2 = lesson_dir / "lesson_2"
lesson_3 = lesson_dir / "lesson_3"
lesson_4 = lesson_dir / "lesson_4"
lesson_5 = lesson_dir / "lesson_5"
lesson_6 = lesson_dir / "lesson_6"
lesson_7 = lesson_dir / "lesson_7"
lesson_8 = lesson_dir / "lesson_8"
lesson_9 = lesson_dir / "lesson_9"

install.mkdir(exist_ok = True, parents=True)
lesson_1.mkdir(exist_ok = True, parents=True)
lesson_2.mkdir(exist_ok = True, parents=True)
lesson_3.mkdir(exist_ok = True, parents=True)
lesson_4.mkdir(exist_ok = True, parents=True)
lesson_5.mkdir(exist_ok = True, parents=True)
lesson_6.mkdir(exist_ok = True, parents=True)
lesson_7.mkdir(exist_ok = True, parents=True)
lesson_8.mkdir(exist_ok = True, parents=True)
lesson_9.mkdir(exist_ok = True, parents=True)

## Create a function to set numeric defaults for simulations

In [4]:
def numeric_default(sim):
    "This sets some default numeric options, these should not be changed unless you know what you are doing"
    #basic solver settings, these values should rarely be changed
    columns = {'GENERAL_RATE_MODEL', 'LUMPED_RATE_MODEL_WITH_PORES', 'LUMPED_RATE_MODEL_WITHOUT_PORES'}
    
    sim.root.input.model.solver.gs_type = 1
    sim.root.input.model.solver.max_krylov = 0
    sim.root.input.model.solver.max_restarts = 10
    sim.root.input.model.solver.schur_safety  = 1e-8
    
    for unit_name, unit in sim.root.input.model.items():
        if 'unit_' in unit_name and unit.unit_type in columns:
            unit.discretization.par_disc_type = 'EQUIDISTANT_PAR'    
            unit.discretization.reconstruction = 'WENO'
            unit.discretization.gs_type = 1
            unit.discretization.max_krylov = 0
            unit.discretization.max_restarts = 10
            unit.discretization.schur_safety = 1.0e-8

            unit.discretization.weno.boundary_model = 0
            unit.discretization.weno.weno_eps = 1e-10
            unit.discretization.weno.weno_order = 3

## Create some support functions to make life a little easier

In [5]:
#This is just a support function to make running the examples a bit easier later
def deleteResultDir(path, match, match_file):
    #This is to try and make sure that only an actual result directory can be removed
    #And only if no other files have been added to the directory, just in case
    if not path.exists():
        return
    allowed = {'evo', 'grad', 'log', 'mcmc', 'meta', 'misc', 'progress', 
               'space', 'error.csv', 'result.h5', 'progress.csv'}
    
    if 'csv' in match:
        allowed.add(match.csv)
    else:
        allowed.add('results.csv')
    
    allowed.add(match_file.name)
    okayToRemove = False
    for child in path.iterdir():
        if child.name not in allowed:
            break
    else:
        okayToRemove = True
    if okayToRemove:
        import shutil
        shutil.rmtree(path)
        
def fractionate(start_seq, stop_seq, times, values):
    temp = []
    for (start, stop) in zip(start_seq, stop_seq):
        selected = (times >= start) & (times <= stop)
        local_times = times[selected]
        local_values = values[selected]
        
        stop = local_times[-1]
        start = local_times[0]

        temp.append(numpy.trapz(local_values, local_times)/ (stop - start))
    return numpy.array(temp)

def run_match(match_obj, match_file, true_values, delete):
    "create the matching object, run it and then look at output"
    match = Match(match_file.as_posix())

    resultDir = Path(match_obj.baseDir) / match_obj.resultsDir
    print("Your results are located at", resultDir.as_posix(), '\n\n')

    #If you want to delete the fitting process to start over remove the # from the next line and re-run this cell\
    if delete:
        deleteResultDir(resultDir, match_obj, match_file)

    match.start_sim()

    match.plot_best()
    best ,score, best_score = match.get_best()
    for key,value in best_score.items():
        print("Method: %s" % key)
        for name, val, true_value in zip(match.cache.parameter_headers, value, true_values):
            print("%s = %.2e  true_value = %.2e error: %.2g%%" % (name, val, true_value, numpy.abs(val-true_value)/val*100))
        print('\n')
    return match

def simple_run_match(match_obj, match_file, delete):
    "create the matching object, run it and then look at output"
    import matplotlib
    
    match = Match(match_file.as_posix())
    
    resultDir = Path(match_obj.baseDir) / match_obj.resultsDir

    #If you want to delete the fitting process to start over remove the # from the next line and re-run this cell\
    if delete:
        deleteResultDir(resultDir, match_obj, match_file)

    match.start_sim()
    
    return match